In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve

sess = sagemaker.Session()
role = get_execution_role()
bucket = sess.default_bucket()
training_image = retrieve("image-classification", boto3.Session().region_name, "1")

### Import Data from S3

In [ ]:
s3_train_key = "images/train"
s3_validation_key = "images/test"

# Retrieve data from S3
s3_train_data = "s3://{}/{}/".format(bucket, s3_train_key)
s3_validation_data = "s3://{}/{}/".format(bucket, s3_validation_key)

# Path where tuned hyperparameters are stored after fitting
s3_output_key = "image-classification-full-training/output"
s3_output = "s3://{}/{}/".format(bucket, s3_output_key)

s3_input_train = sagemaker.TrainingInput(
    s3_data=s3_train_data, content_type="image/jpeg"
)
s3_input_validation = sagemaker.TrainingInput(
    s3_data=s3_validation_data, content_type="image/jpeg"
)

### Setting parameters

In [ ]:
sess = sagemaker.Session()
imageclassification = sagemaker.estimator.Estimator(
    training_image,
    role,
    instance_count=1,
    instance_type="ml.m5.4xlarge",
    output_path=s3_output,
    sagemaker_session=sess,
)

imageclassification.set_hyperparameters(
    num_layers=18,
    image_shape="3,48,48",
    num_classes=7,
    epochs=25,
    top_k="2",
    num_training_samples=15420,
    precision_dtype="float32",
    augmentation_type="crop_color_transform",
)

### Hyperparmeters

In [ ]:
from time import gmtime, strftime
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

tuning_job = "imageclassif-job-{}".format(strftime("%d-%H-%M-%S", gmtime()))

hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.0001, 0.1),
    "mini_batch_size": IntegerParameter(16, 64),
    "optimizer": CategoricalParameter(["sgd", "adam", "rmsprop", "nag"]),
}

objective_metric_name = "validation:accuracy"

tuner = HyperparameterTuner(
    imageclassification,
    objective_metric_name,
    hyperparameter_ranges,
    objective_type="Maximize",
    max_jobs=10,
    max_parallel_jobs=2,
    early_stopping_type="Auto",
)

### Training

In [ ]:
tuner.fit(
    {"train": s3_input_train, "validation": s3_input_validation},
    job_name=tuning_job,
    include_cls_metadata=False,
)
tuner.wait()

### Top 5 training jobs

In [ ]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job)
tuner_metrics.dataframe().sort_values(["FinalObjectiveValue"], ascending=False).head(5)